# Question 1

Importing the DataSet

In [1]:
import tensorflow_datasets as tfds
import numpy as np
import cv2
from skimage.feature import hog
from sklearn import svm
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA


In [4]:
train_ds, test_ds = tfds.load('cifar10', split=['train','test'], as_supervised=True, batch_size = -1)

2023-10-25 13:52:31.787353: I tensorflow/core/util/port.cc:111] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-10-25 13:52:31.788744: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-10-25 13:52:31.809119: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-10-25 13:52:31.809144: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-10-25 13:52:31.809159: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to regi

Dl Completed...: 0 url [00:00, ? url/s]

Dl Size...: 0 MiB [00:00, ? MiB/s]

Extraction completed...: 0 file [00:00, ? file/s]

Generating splits...:   0%|          | 0/2 [00:00<?, ? splits/s]

Generating train examples...:   0%|          | 0/50000 [00:00<?, ? examples/s]

Shuffling /home/iiserb/tensorflow_datasets/cifar10/3.0.2.incompleteAOGWTF/cifar10-train.tfrecord*...:   0%|   …

Generating test examples...:   0%|          | 0/10000 [00:00<?, ? examples/s]

Shuffling /home/iiserb/tensorflow_datasets/cifar10/3.0.2.incompleteAOGWTF/cifar10-test.tfrecord*...:   0%|    …

Dataset cifar10 downloaded and prepared to /home/iiserb/tensorflow_datasets/cifar10/3.0.2. Subsequent calls will reuse this data.


In [5]:
train_images, train_labels = tfds.as_numpy(train_ds)
test_images, test_labels = tfds.as_numpy(test_ds)

Normalizing the Image Data

In [6]:
X_train, X_test = train_images, test_images
y_train, y_test = train_labels, test_labels

**Subpart 2**\
HoG + SVM

In [7]:
def extract_hog_features(X):
    features = []
    for image in X:
        # Convert image to grayscale
        gray_image = cv2.cvtColor(image, cv2.COLOR_RGB2GRAY)

        # Ensure the image has the correct data type (CV_8U)
        gray_image = np.uint8(gray_image * 255)

        # Calculate HOG features
        fd = hog(gray_image, pixels_per_cell=(8, 8), cells_per_block=(2, 2), block_norm='L2-Hys')
        features.append(fd)
    return np.array(features)

In [8]:
X_train_hog = extract_hog_features(X_train)
X_test_hog = extract_hog_features(X_test)

In [9]:
scaler = StandardScaler()
X_train_hog = scaler.fit_transform(X_train_hog)
X_test_hog = scaler.transform(X_test_hog)


In [10]:
xt, xte, yt, yte = train_test_split(X_train_hog, y_train, test_size = 0.2, random_state = 42)

In [11]:
clf = svm.SVC(kernel='linear', C=1)
clf.fit(xt, yt)


SVC(C=1, kernel='linear')

In [12]:
y_pred = clf.predict(xte)
accuracy = accuracy_score(yte, y_pred)
print("Accuracy: {:.2f}%".format(accuracy * 100))


Accuracy: 51.72%


In [13]:
y_pred

array([3, 1, 3, ..., 5, 5, 7])

*Building the Classification_Report*

In [ ]:
from sklearn.metrics import classification_report

print(classification_report(yte, y_pred))

*Building the Confusion Matrix*

In [ ]:
from sklearn.metrics import confusion_matrix
import seaborn as sns

conf_mat = confusion_matrix(yte, y_pred)
sns.heatmap(conf_mat, annot = True)